In [29]:
!pip install transformers -U

In [30]:
!pip install transformers[torch]

In [31]:
!pip install accelerate -U

In [32]:
import pandas as pd

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Abusive_Comment_Detector/Dataset/telugu-hate-speech-train.csv", engine="python")
data

,Data,Sentiment
0,thappu chesina vaallaku vanike kaadu inka anni...,0
1,dhusta chaathuryam! meeru ilantivi enni chesi...,0
2,vetakaram super. govt ki siggu seram radu. end...,0
3,only rajakiyam ga vadukovatanike ee dharidrapu...,0
4,katam hogaya narayana pedda bokada college,0
...,...,...
3995,నిజాయితీపరుడంట....same బాస్ క్వాలిటీస్ చాలా ఉన...,0
3996,ఓహో ఈ నిజాయితీపరుడైన రవి ప్రకాష్ లో పవన్ లక్షణ...,0
3997,వీడీ పేరు రాంగోపాల్ వర్మ! వీడి చేతిలో పడ్డ అమ్...,0
3998,నువ్వు మాత్రం డబ్బులేస్తే ముష్టి వాడి పక్కన అయ...,0


In [35]:
data['Sentiment'].value_counts()

1    2061
0    1939
Name: Sentiment, dtype: int64

In [36]:
data = data[['Data','Sentiment']]
data = data[0:1000]
data

,Data,Sentiment
0,thappu chesina vaallaku vanike kaadu inka anni...,0
1,dhusta chaathuryam! meeru ilantivi enni chesi...,0
2,vetakaram super. govt ki siggu seram radu. end...,0
3,only rajakiyam ga vadukovatanike ee dharidrapu...,0
4,katam hogaya narayana pedda bokada college,0
...,...,...
995,"janasena vallaki subject ledu, ye debate aina ...",1
996,​ycp rip jai janasena jai pspk,1
997,4 lakh crore apu cheshadu yekada poyayi sigu l...,0
998,​wow pawan jee what a idia ur running mak govt...,1


In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [38]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [40]:
model = model.to('cuda')

In [41]:
sample_data = ["katam hogaya narayana pedda bokada college","నువ్వు మాత్రం డబ్బులేస్తే ముష్టి వాడి పక్కన"]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 29354, 2213, 27589, 12186, 24331, 2050, 21877, 25062, 8945, 2912, 2850, 2267, 102], [101, 100, 100, 100, 100, 100, 100, 102, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [42]:
X = list(data["Data"])
y = list(data["Sentiment"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [43]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [44]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [45]:
len(X_train),len(X_val)

(800, 200)

In [46]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [47]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [48]:
train_dataset[5]

{'input_ids': tensor([  101, 22195,  7911,  3654,  4305,  2175, 14343, 23451,  2072, 13642,
         21559, 22794, 13944,  2632, 11663,  5162,  2419,  6979,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [49]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [50]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=19,
    per_device_train_batch_size=16

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [51]:
trainer.train()

Step,Training Loss
500,0.317800


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=950, training_loss=0.22570851777729234, metrics={'train_runtime': 355.2883, 'train_samples_per_second': 42.782, 'train_steps_per_second': 2.674, 'total_flos': 1015444229280000.0, 'train_loss': 0.22570851777729234, 'epoch': 19.0})

In [52]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.814652681350708,
 'eval_accuracy': 0.695,
 'eval_precision': 0.6886792452830188,
 'eval_recall': 0.7227722772277227,
 'eval_f1': 0.7053140096618357,
 'eval_runtime': 1.2916,
 'eval_samples_per_second': 154.843,
 'eval_steps_per_second': 19.355,
 'epoch': 19.0}

In [53]:
text = "​ycp rip jai janasena jai pspk"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-3.9772,  3.8057]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[4.1666e-04, 9.9958e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[4.1666065e-04, 9.9958330e-01]], dtype=float32)

In [54]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/Abusive_Comment_Detector')

In [55]:
model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Abusive_Comment_Detector")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [57]:
# text = "That was good point"
text = "కొన్ని ఆంధ్ర పందులకి బురదలోనే (ycp)ఆనందం...ఏం చేస్తాం పంది జాతి కదా అట్లాగే ఉంటది"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.997615  , 0.00238506]], dtype=float32)